In [307]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from scipy.stats import ttest_rel

In [308]:
#Cargar el dataset
df=pd.read_csv("../data/ml/one_big_table.csv")


In [309]:
# Calcular los promedios para cada 'user_id'
df_ml_mean = df.groupby('user_id').agg({
    'costo_extra_messages': 'mean',
    'costo_extra_minutes': 'mean',
    'costo_extra_mb_used': 'mean',
    'costo_extra': 'mean',
    'exceso_messages': 'mean',
    'exceso_minutes': 'mean',
    'exceso_mb_used': 'mean',
    'total_messages': 'mean',
    'total_minutes': 'mean',
    'total_mb': 'mean'
}).reset_index()

# Calcular las desviaciones estándar para cada 'user_id'
df_ml_std = df.groupby('user_id').agg({
    'costo_extra_messages': 'std',
    'costo_extra_minutes': 'std',
    'costo_extra_mb_used': 'std',
    'costo_extra': 'std',
    'exceso_messages': 'std',
    'exceso_minutes': 'std',
    'exceso_mb_used': 'std',
    'total_messages': 'std',
    'total_minutes': 'std',
    'total_mb': 'std'
}).reset_index()

# Unir los resultados de los promedios y desviaciones estándar con el df original
df = df.merge(df_ml_mean, on='user_id', how='left', suffixes=('', '_promedio'))
df = df.merge(df_ml_std, on='user_id', how='left', suffixes=('', '_std'))
df.head()


,month,year,user_id,total_minutes,total_messages,total_mb,first_name,last_name,age,city,...,costo_extra_messages_std,costo_extra_minutes_std,costo_extra_mb_used_std,costo_extra_std,exceso_messages_std,exceso_minutes_std,exceso_mb_used_std,total_messages_std,total_minutes_std,total_mb_std
0,0,0,1003,0.00,0.0,0.00,Reynaldo,Jenkins,52,"Tulsa, OK MSA",...,0.0000,3.875377,18.365425,22.240803,0.00000,129.179236,1836.542546,14.433757,273.516803,6270.592613
1,0,0,1011,0.00,0.0,0.00,Halina,Henry,73,"Cleveland-Elyria, OH MSA",...,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,29.351966,198.154700,10052.130289
2,0,0,1019,0.00,0.0,0.00,Shizue,Landry,34,"Jacksonville, FL MSA",...,0.6755,0.000000,24.895084,25.570584,22.51666,0.000000,2489.508380,36.970094,109.228117,6899.076847
3,0,0,1029,0.00,0.0,0.00,Franklyn,Henson,59,"Tampa-St. Petersburg-Clearwater, FL MSA",...,0.0000,1.691432,13.569880,14.305554,0.00000,56.381080,1356.988027,6.012613,295.013505,7764.526504
4,0,0,1042,106.83,0.0,1854.93,Clementina,Mclaughlin,40,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA",...,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,65.877368,2510.185850


In [310]:
#Dropear las columnas no utiles para el modelo como user_id
#o que se derivan del precio de los servicios
#como costo_extra_messages, costo_extra_minutes, costo_extra_mb_used, costo_extra
df.drop(columns=["user_id","first_name","last_name",'churn_date','churn_month','churn_year','reg_date','reg_month','reg_year'
                 ,'usd_per_mb','usd_per_minute','usd_per_message','usd_monthly_pay','mb_per_month_included','minutes_included','messages_included',
                 'city','costo_extra_messages','costo_extra_minutes','costo_extra_mb_used','costo_extra',
                    'total_messages','total_minutes','total_mb','exceso_mb_used','exceso_minutes','exceso_messages'],inplace=True)
print(df.columns)


Index(['month', 'year', 'age', 'plan_name', 'total_cost', 'city_code',
       'costo_extra_messages_promedio', 'costo_extra_minutes_promedio',
       'costo_extra_mb_used_promedio', 'costo_extra_promedio',
       'exceso_messages_promedio', 'exceso_minutes_promedio',
       'exceso_mb_used_promedio', 'total_messages_promedio',
       'total_minutes_promedio', 'total_mb_promedio',
       'costo_extra_messages_std', 'costo_extra_minutes_std',
       'costo_extra_mb_used_std', 'costo_extra_std', 'exceso_messages_std',
       'exceso_minutes_std', 'exceso_mb_used_std', 'total_messages_std',
       'total_minutes_std', 'total_mb_std'],
      dtype='object')


In [311]:
print(len(df))
df.head()



3287


,month,year,age,plan_name,total_cost,city_code,costo_extra_messages_promedio,costo_extra_minutes_promedio,costo_extra_mb_used_promedio,costo_extra_promedio,...,costo_extra_messages_std,costo_extra_minutes_std,costo_extra_mb_used_std,costo_extra_std,exceso_messages_std,exceso_minutes_std,exceso_mb_used_std,total_messages_std,total_minutes_std,total_mb_std
0,0,0,52,0,20.0,28,0.000,1.118725,5.301642,6.420367,...,0.0000,3.875377,18.365425,22.240803,0.00000,129.179236,1836.542546,14.433757,273.516803,6270.592613
1,0,0,73,1,70.0,26,0.000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,29.351966,198.154700,10052.130289
2,0,0,34,0,20.0,6,0.195,0.000000,7.186592,7.381592,...,0.6755,0.000000,24.895084,25.570584,22.51666,0.000000,2489.508380,36.970094,109.228117,6899.076847
3,0,0,59,0,20.0,6,0.000,1.014400,4.241108,5.255508,...,0.0000,1.691432,13.569880,14.305554,0.00000,56.381080,1356.988027,6.012613,295.013505,7764.526504
4,0,0,40,0,20.0,31,0.000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,65.877368,2510.185850


In [312]:
#Valores nulos
df.isnull().sum()


month                             0
year                              0
age                               0
plan_name                         0
total_cost                        0
city_code                         0
costo_extra_messages_promedio     0
costo_extra_minutes_promedio      0
costo_extra_mb_used_promedio      0
costo_extra_promedio              0
exceso_messages_promedio          0
exceso_minutes_promedio           0
exceso_mb_used_promedio           0
total_messages_promedio           0
total_minutes_promedio            0
total_mb_promedio                 0
costo_extra_messages_std         34
costo_extra_minutes_std          34
costo_extra_mb_used_std          34
costo_extra_std                  34
exceso_messages_std              34
exceso_minutes_std               34
exceso_mb_used_std               34
total_messages_std               34
total_minutes_std                34
total_mb_std                     34
dtype: int64

In [313]:
#Reemplazar los nulos por 0
df.fillna(0,inplace=True)

Se asume una distribucion normal por la cantidad de data, por lo que se hace un standardScaler. Sin embargo, solo se escalan datos de costos, etc.

In [314]:
#Conjuntos de entrenamiento y prueba, se quita el indice de pandas

X =df.drop(columns=["total_cost"]).reset_index(drop=True)
y = df["total_cost"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
#Eliminar indices
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)
#Escalado de datos
scaler = StandardScaler()
#Se escalaran las ccolumnas total_minutes,total_messages,total_mb_used,exceso_messages,exceso_minutes,exceso_mb_used
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#Convertir a arreglos numpy y_train y y_test
y_train = y_train.values
y_test = y_test.values
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[ 1.27867346  0.          0.92226393 ...  0.07329264  1.10426235
   0.13172174]
 [-0.09284901  0.         -1.65024568 ... -0.96962727  0.73390562
   0.2685732 ]
 [-1.46437148  0.          1.10174134 ... -0.87121771 -0.76901995
  -1.05094901]
 ...
 [-0.09284901  0.          0.14452847 ... -0.96962727  0.89664835
   1.35548722]
 [-0.77861024  0.          0.38383168 ...  0.19469921  0.3825281
   0.09799037]
 [ 1.27867346  0.         -0.6930328  ...  3.26762086  3.68510345
   7.11352308]]
[[ 5.92912223e-01  0.00000000e+00 -2.14426363e-01 ...  1.91513994e-02
   6.26705642e-02  8.56216924e-01]
 [-7.78610244e-01  0.00000000e+00  6.82960708e-01 ...  3.01963635e-01
  -1.52409441e+00  3.39043138e-01]
 [ 5.92912223e-01  0.00000000e+00 -1.11181343e+00 ...  4.43702700e-01
   6.24083700e-01  9.38816566e-01]
 ...
 [ 2.50031606e-01  0.00000000e+00 -1.53059407e+00 ... -8.54381789e-01
  -1.18381565e-01 -3.28462175e-01]
 [-7.78610244e-01  0.00000000e+00  9.22263927e-01 ...  5.79392714e-01
   2.69440268e

In [315]:
def obtener_scores(modelo, X_train, y_train, X_test, y_test):
    y_pred_train = modelo.predict(X_train)
    y_pred_test = modelo.predict(X_test)
    rmse_train= np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test= np.sqrt(mean_squared_error(y_test, y_pred_test))
    mae_train= mean_absolute_error(y_train, y_pred_train)
    mae_test= mean_absolute_error(y_test, y_pred_test)
    r2_train= r2_score(y_train, y_pred_train)
    r2_test= r2_score(y_test, y_pred_test)
    salida= {"RMSE_train":rmse_train,"MAE_train":mae_train,"R2_train":r2_train,"RMSE_test":rmse_test,"MAE_test":mae_test,"R2_test":r2_test}
    return salida

In [316]:
#Regresión lineal usando scikit-learn LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#Scores
print(obtener_scores(model, X_train, y_train, X_test, y_test))



{'RMSE_train': 23.1441179195567, 'MAE_train': 12.489705878244967, 'R2_train': 0.5302277299730787, 'RMSE_test': 24.399318782944153, 'MAE_test': 12.922741292440557, 'R2_test': 0.49872265680924754}


In [317]:
#Regresión lineal con SGDRegressor que use regularización L1 y K-Fold Cross Validation para hallar el mejor valor de alpha
# y learning rate (eta0)
from sklearn.model_selection import GridSearchCV
params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
          'eta0': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
sgd = SGDRegressor(penalty='l1', max_iter=1000)
grid = GridSearchCV(sgd, params, cv=5)
grid.fit(X_train, y_train)
print(grid.best_params_)
model = grid.best_estimator_
model.fit(X_train, y_train)
#Scores
obtener_scores(model, X_train, y_train, X_test, y_test)



{'alpha': 1, 'eta0': 0.01}


{'RMSE_train': 23.236565053100456,
 'MAE_train': 12.396756526689414,
 'R2_train': 0.5264673069762738,
 'RMSE_test': 24.45489880458725,
 'MAE_test': 12.67178036716582,
 'R2_test': 0.49643630294924246}

In [318]:
#Regresión polinomial usando PolynomialFeatures, Stochastic Gradient Descent, elastic net
# y K-Fold Cross Validation para hallar el mejor valor de alpha y el mejor grado del polinomio
#Para todo esto también se debe usar un pipeline para encadenar los procesos e incluir PolynomialFeatures en el modelo
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
params = {'polynomialfeatures__degree': [1, 2, 3], 'sgdregressor__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],
          'sgdregressor__eta0': [0.01, 0.1, 1, 10]}
model= make_pipeline(PolynomialFeatures(), SGDRegressor(penalty='elasticnet', max_iter=100))
grid = GridSearchCV(model, params, cv=2,verbose=1)
grid.fit(X_train, y_train)
print(grid.best_params_)
model = grid.best_estimator_
model.fit(X_train, y_train)
#Scores
obtener_scores(model, X_train, y_train, X_test, y_test)

Fitting 2 folds for each of 72 candidates, totalling 144 fits
{'polynomialfeatures__degree': 1, 'sgdregressor__alpha': 0.1, 'sgdregressor__eta0': 0.01}


{'RMSE_train': 23.382886988872194,
 'MAE_train': 13.258198376709315,
 'R2_train': 0.5204848068938402,
 'RMSE_test': 24.774852028970443,
 'MAE_test': 13.599737510361601,
 'R2_test': 0.48317345464948724}

In [321]:
# Ajustar el modelo de LinearRegression
model1 = LinearRegression()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)
print(obtener_scores(model1, X_train, y_train, X_test, y_test))
# Ajustar el modelo de PolynomialFeatures con SGDRegressor
model2 = make_pipeline(PolynomialFeatures(degree=1), SGDRegressor(penalty='elasticnet', max_iter=10000, alpha=0.1, eta0=0.01))
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
print(obtener_scores(model2, X_train, y_train, X_test, y_test))

# Realizar la prueba t de hipótesis: Evaluar la diferencia entre los modelos
# Hipótesis nula: modelo1 es mejor o igual que modelo2 (si y_pred1 >= y_pred2)
# Comparar las diferencias entre las predicciones de ambos modelos
differences = y_pred1 - y_pred2

# Realizar la prueba t de hipótesis sobre las diferencias
t_stat, p_value = ttest_rel(differences, [0] * len(differences))  # Comparando las diferencias con 0

# Imprimir el resultado de la prueba t
print(f"Estadístico t: {t_stat}")
print(f"Valor p: {p_value}")

# Conclusión según el valor p
if p_value < 0.05:
    print("Rechazamos la hipótesis nula: el modelo con PolynomialFeatures y SGDRegressor es significativamente mejor.")
else:
    print("No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que el modelo con PolynomialFeatures y SGDRegressor es mejor.")


{'RMSE_train': 23.1441179195567, 'MAE_train': 12.489705878244967, 'R2_train': 0.5302277299730787, 'RMSE_test': 24.399318782944153, 'MAE_test': 12.922741292440557, 'R2_test': 0.49872265680924754}
{'RMSE_train': 23.430258389644326, 'MAE_train': 12.710090698772305, 'R2_train': 0.5185399387623513, 'RMSE_test': 24.605733463926942, 'MAE_test': 12.89843928241567, 'R2_test': 0.49020531428790015}
Estadístico t: 1.1410844224098455
Valor p: 0.2546672229440891
No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que el modelo con PolynomialFeatures y SGDRegressor es mejor.


In [322]:
#Score final
obtener_scores(model1, X_train, y_train, X_test, y_test)

{'RMSE_train': 23.1441179195567,
 'MAE_train': 12.489705878244967,
 'R2_train': 0.5302277299730787,
 'RMSE_test': 24.399318782944153,
 'MAE_test': 12.922741292440557,
 'R2_test': 0.49872265680924754}